In [1]:
# import scikit-allel
import allel
# check which version is installed
print(allel.__version__)

1.3.5


In [2]:
## This data is in some weird vcf format
## Found a package above to deal with this

In [3]:
callset = allel.read_vcf(r"D:\Maize_Comp\Training_Data\5_Genotype_Data_All_Years.vcf")

In [4]:
# These are the keys
sorted(callset.keys())

['calldata/GT',
 'samples',
 'variants/ALT',
 'variants/CHROM',
 'variants/FILTER_PASS',
 'variants/ID',
 'variants/POS',
 'variants/QUAL',
 'variants/REF']

In [5]:
# basically these samples are the Hybrids that we see in train and test
callset['samples']

array(['2369/DK3IIH6', '2369/PHN82', '2369/PHZ51', ...,
       'Z038E0057/DK3IIH6', 'Z038E0057/LH162', 'Z038E0057/PHZ51'],
      dtype=object)

In [6]:
# How many such Hybrids?
len(callset['samples'])

4928

In [7]:
# These is where the features of the Hybrids will come from
# So basically you can see we have 437214 x 2 dimensional features
# for all these 4928 Hybrids
callset['calldata/GT'].shape

(437214, 4928, 2)

In [8]:
first_col = callset['calldata/GT'][:,:,0].T

In [9]:
second_col = callset['calldata/GT'][:,:,1].T

In [10]:
import numpy as np

In [11]:
np.save("first_col.npy", first_col)

In [12]:
np.save("second_col.npy", second_col)

In [13]:
np.save("hybrid_order.npy", callset['samples'])

In [14]:
from scipy.sparse import csr_matrix

In [15]:
# first_col = csr_matrix(first_col)

In [16]:
# second_col = csr_matrix(second_col)

In [17]:
from sklearn.decomposition import PCA

In [1]:
second_col.shape

NameError: name 'second_col' is not defined

In [18]:
first_col.shape

(4928, 437214)

In [19]:
pca_size = 200

In [20]:
mds_1 = PCA(n_components=pca_size)

In [21]:
first_col_mds = mds_1.fit_transform(first_col)

In [22]:
mds_2 = PCA(n_components=pca_size)

In [23]:
second_col_mds = mds_2.fit_transform(second_col)

In [24]:
first_col_mds.shape

(4928, 200)

In [25]:
second_col_mds.shape

(4928, 200)

In [26]:
np.save("first_col_pca_" + str(pca_size) + ".npy", first_col_mds)

In [27]:
np.save("second_col_pca_" + str(pca_size) + ".npy", second_col_mds)

In [28]:
import tensorflow as tf

In [29]:
padding_vector = np.zeros((1, pca_size))
unknown_vector = np.zeros((1, pca_size))
weight_vectors = np.vstack((padding_vector, unknown_vector))
weight_vectors = np.vstack((weight_vectors, first_col_mds))
embedding_layer_first_col = tf.keras.layers.Embedding(len(weight_vectors),
                            weight_vectors.shape[1],
                            weights=[weight_vectors],
                            mask_zero = False,
                            trainable=False)

In [30]:
vectorize_layer_first_col = tf.keras.layers.TextVectorization(
                     output_mode='int',
                     vocabulary=callset['samples'], 
                     standardize = None)

In [31]:
input_layer = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

vectorize = vectorize_layer_first_col(input_layer)

vectorize.shape


TensorShape([None, None])

In [32]:
emb_output = embedding_layer_first_col(vectorize)

In [33]:
emb_output

<KerasTensor: shape=(None, None, 200) dtype=float32 (created by layer 'embedding')>

In [34]:
first_col_pca_model = tf.keras.models.Model(input_layer, emb_output)

In [35]:
first_col_pca_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 200)         986000    
                                                                 
Total params: 986,000
Trainable params: 0
Non-trainable params: 986,000
_________________________________________________________________


In [36]:
first_col_pca_model.save("first_col_pca_" + str(pca_size))

INFO:tensorflow:Assets written to: first_col_pca_200\assets


In [37]:
first_col_pca_model = tf.keras.models.load_model("first_col_pca_" + str(pca_size))

In [38]:
# first_col_mds

In [39]:
padding_vector = np.zeros((1, pca_size))
unknown_vector = np.zeros((1, pca_size))
weight_vectors = np.vstack((padding_vector, unknown_vector))
weight_vectors = np.vstack((weight_vectors, second_col_mds))
embedding_layer_second_col = tf.keras.layers.Embedding(len(weight_vectors),
                            weight_vectors.shape[1],
                            weights=[weight_vectors],
                            mask_zero = False,
                            trainable=False)

In [40]:
vectorize_layer_second_col = tf.keras.layers.TextVectorization(
                     output_mode='int',
                     vocabulary=callset['samples'], 
                     standardize = None)

In [41]:
callset["samples"]

array(['2369/DK3IIH6', '2369/PHN82', '2369/PHZ51', ...,
       'Z038E0057/DK3IIH6', 'Z038E0057/LH162', 'Z038E0057/PHZ51'],
      dtype=object)

In [42]:
input_layer = tf.keras.layers.Input(shape = (1,), dtype = tf.string)

vectorize = vectorize_layer_second_col(input_layer)

vectorize.shape


TensorShape([None, None])

In [43]:
emb_output = embedding_layer_second_col(vectorize)

In [44]:
emb_output

<KerasTensor: shape=(None, None, 200) dtype=float32 (created by layer 'embedding_1')>

In [45]:
second_col_pca_model = tf.keras.models.Model(input_layer, emb_output)

In [46]:
second_col_pca_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, None, 200)         986000    
                                                                 
Total params: 986,000
Trainable params: 0
Non-trainable params: 986,000
_________________________________________________________________


In [47]:
second_col_pca_model.save("second_col_pca_" + str(pca_size))

INFO:tensorflow:Assets written to: second_col_pca_200\assets
